In [28]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
import requests
from io import BytesIO
from PIL import Image
import json
import zipfile

In [39]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("UTKFace/utkface-new")

# print("Path to dataset files:", path)

In [29]:
def download_utkface_from_kaggle():
  if not os.path.exists("UTKFace"):
    print("pobieranie danych UTK z kaggle...")
    os.system("kaggle datasets download -d jangedoo/utkface-dataset")
    print("rozpakowanie danych UTK...")
    # Change 'utkface-new.zip' to 'utkface-dataset.zip' to match the downloaded filename
    with zipfile.ZipFile("utkface-dataset.zip", "r") as zip_ref:
        zip_ref.extractall("UTKFace")
    print("usuwanie zip UTK...")
    os.remove("utkface-dataset.zip")
    print("Dane gotowe....")
  else:
    print("Dane UTK już istnieją")

In [34]:
# === 2. Wczytywanie danych ===
def load_utkface_dataset(path, image_size=(64, 64), limit=None):
    images, ages = [], []
    files = os.listdir(path)
    # if limit:  # Comment out or remove this line to load all images
    #     files = files[:limit]
    for filename in files:
        if filename.endswith('.jpg'):
            try:
                age = int(filename.split('_')[0])
                img_path = os.path.join(path, filename)
                img = cv2.imread(img_path)
                # Check if image was loaded correctly
                if img is None:
                    print(f"Failed to load image: {img_path}")
                    continue
                img = cv2.resize(img, image_size)
                images.append(img)
                ages.append(age)
            except Exception as e:
                print(f"Błąd przy pliku {filename}: {e}")
    return np.array(images), np.array(ages)

In [31]:
def create_age_regression_model(input_shape=(64, 64, 3)):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D(2, 2),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D(2, 2),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D(2, 2),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(1)  # regresja wieku
    ])
    return model

In [32]:
def predict_age_from_url(image_url, model, image_size=(64, 64)):
    try:
        response = requests.get(image_url)
        img_pil = Image.open(BytesIO(response.content)).convert('RGB')
        img_pil = img_pil.resize(image_size)
        img_np = np.array(img_pil) / 255.0
        img_input = np.expand_dims(img_np, axis=0)
        predicted_age = model.predict(img_input)[0][0]
        plt.imshow(img_np)
        plt.title(f"Przewidywany wiek: {predicted_age:.1f} lat")
        plt.axis('off')
        plt.show()
        return predicted_age
    except Exception as e:
        print(f"Błąd przy przetwarzaniu obrazu: {e}")
        return None

In [43]:
# Pobranie zbioru z Kaggle
# download_utkface_from_kaggle()

# Wczytanie i przygotowanie danych

zip_path = "archive.zip"
extract_path = "UTKFace"

os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Dane rozpakowane....")

print("Wczytywanie danych...")
x, y = load_utkface_dataset('UTKFace', image_size=(64, 64), limit=10000)
x = x / 255.0
y = y.reshape(-1, 1)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Budowa i trening modelu
print("Trening modelu...")
model = create_age_regression_model()
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
model.fit(x_train, y_train, validation_split=0.1, epochs=10, batch_size=32)

# Ewaluacja
loss, mae = model.evaluate(x_test, y_test)
print(f"\nTest MAE (wiek): {mae:.2f} lat")

# Predykcja z URL
test_url = "UTKFace/100_0_0_20170112215240346.jpg.chip.jpg"
print("\nPredykcja wieku z obrazu URL:")
predicted = predict_age_from_url(test_url, model)
print(f"Model oszacował wiek na: {predicted:.1f} lat")

Wczytywanie danych...
Trening modelu...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
243/534 ━━━━━━━━━━━━━━━━━━━━ 1:28 306ms/step - loss: 567.6722 - mae: 18.3258

KeyboardInterrupt: 